In [1]:
import pandas as pd
data = pd.read_csv("HDFS_2k.log_structured.csv")
data.drop(['LineId'])

In [2]:
data.head(5)

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate
0,1,81109,203615,148,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_38865049064139...,E10,PacketResponder <*> for block blk_<*> terminating
1,2,81109,203807,222,INFO,dfs.DataNode$PacketResponder,PacketResponder 0 for block blk_-6952295868487...,E10,PacketResponder <*> for block blk_<*> terminating
2,3,81109,204005,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.addStoredBlock: blockMap upd...,E6,BLOCK* NameSystem.addStoredBlock: blockMap upd...
3,4,81109,204015,308,INFO,dfs.DataNode$PacketResponder,PacketResponder 2 for block blk_82291938032499...,E10,PacketResponder <*> for block blk_<*> terminating
4,5,81109,204106,329,INFO,dfs.DataNode$PacketResponder,PacketResponder 2 for block blk_-6670958622368...,E10,PacketResponder <*> for block blk_<*> terminating


In [26]:
data['Level'].groupby(data['Level']).count()

Level
INFO    1920
WARN      80
Name: Level, dtype: int64

In [37]:
data['Date'].groupby(data['Date']).count()

Date
81109    150
81110    965
81111    885
Name: Date, dtype: int64

In [21]:
len(data['LineId'].unique())

2000

In [53]:
data['Pid'].groupby(data['Pid']).count()

Pid
13        20
18        28
19       242
26        70
27        84
        ... 
26494      1
26526      1
26527      1
26595      1
26895      1
Name: Pid, Length: 1054, dtype: int64

In [47]:
blk=[]
for i in range(len(data['Pid'])):
    id_text=(data['Content'][i].split('blk_', 1)[1])
    id_text="blk_"+id_text
    blk.append(id_text.split('terminating', 1)[0])
data['BlockId']=blk

In [35]:
label_data=pd.read_csv('anomaly_label.csv')
label_data['Label'].groupby(label_data['Label']).count()

In [50]:
HDFS_with_label = data.merge(label_data, on='BlockId')
HDFS_with_label.to_csv("HDFS_with_label.csv", index=False)

In [57]:
merged['BlockId'].groupby(merged['BlockId']).count()

In [69]:
len(data)

2000

In [61]:
e[3]>1

False

In [2]:
import trypro as pss

ModuleNotFoundError: No module named 'torch'

In [76]:
conda install pytorch-cpu torchvision-cpu -c pytorch

^C

Note: you may need to restart the kernel to use updated packages.


In [ ]:
conda create -n env_pytorch python=3.6

In [ ]:
conda install -c pytorch pytorch